In [1]:
using Flux
using Statistics

In [2]:
###Complex Weight Initialization
function c_uniform(in::Int64, out::Int64)
    return Matrix{Float32}(Flux.glorot_uniform(in,out)/sqrt(2)), Matrix{Float32}(Flux.glorot_uniform(in,out)/sqrt(2))
end

struct CDense{F, M<:AbstractMatrix, M<:AbstractMatrix, B}
  weight::M ###abuse of notation to use flux conversion
  weight_imag::M
  bias::B
  σ::F
  function CDense(weight::M,weight_imag::M,bias = true, σ::F = identity) where {M<:AbstractMatrix, F}
    W = [weight -weight_imag; weight_imag weight]
    b = Flux.create_bias(W, bias, size(W,1))
    new{F,M,M,typeof(b)}(weight,weight_imag, b, σ)
  end
end

function CDense((in, out)::Pair{<:Integer, <:Integer}, σ = identity;
               init = c_uniform, bias = true)
  CDense(init(out, in)..., bias, σ)
end

function (a::CDense)(x::AbstractVecOrMat)
  Flux._size_check(a, x, 1 => size(a.weight, 2)+size(a.weight_imag, 2))
  σ = NNlib.fast_act(a.σ, x)  # replaces tanh => tanh_fast, etc
  xT = Flux._match_eltype(a, x)  # fixes Float64 input, etc.
  return σ.([a.weight -a.weight_imag; a.weight_imag a.weight] * xT .+ a.bias)
end

function (a::CDense)(x::AbstractArray)
  Flux._size_check(a, x, 1 => size(a.weight, 2)+size(a.weight_imag, 2))
  reshape(a(reshape(x, size(x,1), :)), :, size(x)[2:end]...)
end

a_sigmoid = x-> Float32(a) * fast_sigmoid(x)

In [11]:
m=CDense(1=>1)

CDense{typeof(identity), Matrix{Float32}, Matrix{Float32}, Vector{Float32}}(Float32[-1.0422541;;], Float32[-1.1857865;;], Float32[0.0, 0.0], identity)

In [12]:
m([1,1])

2-element Vector{Float32}:
  0.1435324
 -2.2280407

In [13]:
Dense(1=>1)([1])

1-element Vector{Float32}:
 1.0359259

LoadError: UndefVarError: `fast_sigmoid_` not defined